In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [2]:
lst = os.listdir()

In [3]:
train_Files = [x for x in lst if x.endswith('_train.csv')]
test_Files = [x for x in lst if x.endswith('_test.csv')]

In [4]:
files = [x[:-10] for x in train_Files]

In [5]:
files = [x for x in files if 'XGB' not in x]

In [6]:
files

['Feat1_LGBM_Gruped_Cats_FE',
 'CGB_Feat5_All_Possible_Feats_5_Fold',
 'Feat_5_LGB_ItemPast_Removed',
 'CGB_Feat5_Extra_Past_Item_5_Fold',
 'LOGITLEVEL1',
 'Feat_5_LGB_Coup_Removed_1337']

In [7]:
Group1 = files
Group2 = [
    'CGB_Feat5_All_Possible_Feats_5_Fold',
    'Feat_1_XGB_OneHot',
    'XGB_PLANE_Feat_5_not_all_one_hot_Coup_Removed',
    'Feat_5_LGB_Coup_Removed'
]

In [8]:
try:
    Group1.remove('LOGITLEVEL1')
    Group1.remove('NN_level1')
    Group1.remove('knnLEVEL1')
except:
    print("Not Found Safe to Proceed.")

Not Found Safe to Proceed.


In [9]:
Group1

['Feat1_LGBM_Gruped_Cats_FE',
 'CGB_Feat5_All_Possible_Feats_5_Fold',
 'Feat_5_LGB_ItemPast_Removed',
 'CGB_Feat5_Extra_Past_Item_5_Fold',
 'Feat_5_LGB_Coup_Removed_1337']

In [10]:
group1Tup = [(x + '_train.csv', x + '_test.csv') for x in Group1]

In [11]:
for x,y in group1Tup:
    assert x in train_Files
    assert y in test_Files

In [12]:
trainGroup1DF = pd.DataFrame()
testGroup1DF = pd.DataFrame()
for trainCol, testCol in group1Tup:
    temp = pd.read_csv(trainCol)
    trainGroup1DF = pd.concat([trainGroup1DF, temp],axis = 1)
    temp = pd.read_csv(testCol)
    testGroup1DF = pd.concat([testGroup1DF, temp],axis = 1)

In [13]:
trainGroup1DF.shape, testGroup1DF.shape

((78369, 5), (50226, 5))

In [14]:
trainCols = set(trainGroup1DF.columns).intersection(testGroup1DF.columns)

In [15]:
## Loading Orig Labels 

train = pd.read_csv('train.csv')
trainGroup1DF['redemption_status'] = train.redemption_status

In [16]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold
gkf = KFold(n_splits=5)
from sklearn.metrics import roc_auc_score
import xgboost
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [17]:
def makeNN():
    model = Sequential()

    model.add(Dense(7, 
                activation='relu',  
                input_dim=5,
                kernel_initializer='uniform'))


    model.add(Dense(9,
                kernel_initializer='uniform',
                activation='relu'))



    model.add(Dense(1,
                kernel_initializer='uniform',
                activation='sigmoid'))
    return model
#model.summary()

In [18]:
oof = np.zeros_like(trainGroup1DF.redemption_status).astype('float')
oot = np.zeros_like(testGroup1DF.index).astype('float')

def getResultSummary(res):
    trainMean = np.mean(list(map(lambda x : x[0], res)))
    validMean = np.mean(list(map(lambda x : x[1], res)))
    trainStd = np.std(list(map(lambda x : x[0], res)))
    validStd = np.std(list(map(lambda x : x[1], res)))
    return {
        
        'Train Mean' : trainMean, 
        'Valid Mean' : validMean, 
        'Train Std'  : trainStd,
        'Valid Std'  : validStd
    }

results = []
featureImp = []
i = 0
for train_index, test_index in gkf.split(trainGroup1DF[trainCols], trainGroup1DF['redemption_status']):
    train_X = trainGroup1DF.iloc[train_index][trainCols]
    valid_X = trainGroup1DF.iloc[test_index][trainCols]
    train_y = trainGroup1DF.iloc[train_index]['redemption_status']
    valid_y = trainGroup1DF.iloc[test_index]['redemption_status']
    model = makeNN()
    sgd = Adam()
    model.compile(optimizer = sgd, 
                   loss = 'binary_crossentropy', 
                   metrics = ['accuracy'])
    model.fit(train_X, train_y, 
               batch_size = 200, 
               epochs = 100, verbose=2)
    valid_pred = model.predict_proba(valid_X)
    train_pred = model.predict_proba(train_X)
    trainRes = roc_auc_score(train_y, train_pred)
    validRes = roc_auc_score(valid_y, valid_pred)
    oof[test_index] = valid_pred.reshape(valid_pred.shape[0], )
    ootPred = model.predict_proba(testGroup1DF[trainCols])
    oot += ootPred.reshape(ootPred.shape[0], ) / 5
    results.append((trainRes, validRes))
    i+=1

Epoch 1/100
 - 0s - loss: 0.3921 - accuracy: 0.9900
Epoch 2/100
 - 0s - loss: 0.0493 - accuracy: 0.9914
Epoch 3/100
 - 0s - loss: 0.0345 - accuracy: 0.9919
Epoch 4/100
 - 0s - loss: 0.0330 - accuracy: 0.9916
Epoch 5/100
 - 0s - loss: 0.0324 - accuracy: 0.9913
Epoch 6/100
 - 0s - loss: 0.0320 - accuracy: 0.9909
Epoch 7/100
 - 0s - loss: 0.0317 - accuracy: 0.9906
Epoch 8/100
 - 0s - loss: 0.0314 - accuracy: 0.9904
Epoch 9/100
 - 0s - loss: 0.0311 - accuracy: 0.9902
Epoch 10/100
 - 0s - loss: 0.0309 - accuracy: 0.9907
Epoch 11/100
 - 0s - loss: 0.0307 - accuracy: 0.9913
Epoch 12/100
 - 0s - loss: 0.0305 - accuracy: 0.9917
Epoch 13/100
 - 0s - loss: 0.0304 - accuracy: 0.9917
Epoch 14/100
 - 0s - loss: 0.0303 - accuracy: 0.9918
Epoch 15/100
 - 0s - loss: 0.0302 - accuracy: 0.9919
Epoch 16/100
 - 0s - loss: 0.0301 - accuracy: 0.9920
Epoch 17/100
 - 0s - loss: 0.0300 - accuracy: 0.9919
Epoch 18/100
 - 0s - loss: 0.0299 - accuracy: 0.9919
Epoch 19/100
 - 0s - loss: 0.0298 - accuracy: 0.9919
Ep

Epoch 56/100
 - 0s - loss: 0.0288 - accuracy: 0.9916
Epoch 57/100
 - 0s - loss: 0.0288 - accuracy: 0.9918
Epoch 58/100
 - 0s - loss: 0.0288 - accuracy: 0.9917
Epoch 59/100
 - 0s - loss: 0.0288 - accuracy: 0.9917
Epoch 60/100
 - 0s - loss: 0.0288 - accuracy: 0.9916
Epoch 61/100
 - 0s - loss: 0.0288 - accuracy: 0.9917
Epoch 62/100
 - 0s - loss: 0.0288 - accuracy: 0.9917
Epoch 63/100
 - 0s - loss: 0.0288 - accuracy: 0.9917
Epoch 64/100
 - 0s - loss: 0.0288 - accuracy: 0.9918
Epoch 65/100
 - 0s - loss: 0.0287 - accuracy: 0.9918
Epoch 66/100
 - 0s - loss: 0.0287 - accuracy: 0.9917
Epoch 67/100
 - 0s - loss: 0.0287 - accuracy: 0.9917
Epoch 68/100
 - 0s - loss: 0.0287 - accuracy: 0.9917
Epoch 69/100
 - 0s - loss: 0.0287 - accuracy: 0.9917
Epoch 70/100
 - 0s - loss: 0.0287 - accuracy: 0.9917
Epoch 71/100
 - 0s - loss: 0.0287 - accuracy: 0.9918
Epoch 72/100
 - 0s - loss: 0.0287 - accuracy: 0.9917
Epoch 73/100
 - 0s - loss: 0.0287 - accuracy: 0.9917
Epoch 74/100
 - 0s - loss: 0.0287 - accuracy: 

Epoch 11/100
 - 0s - loss: 0.0309 - accuracy: 0.9906
Epoch 12/100
 - 0s - loss: 0.0308 - accuracy: 0.9906
Epoch 13/100
 - 0s - loss: 0.0307 - accuracy: 0.9906
Epoch 14/100
 - 0s - loss: 0.0305 - accuracy: 0.9906
Epoch 15/100
 - 0s - loss: 0.0304 - accuracy: 0.9906
Epoch 16/100
 - 0s - loss: 0.0303 - accuracy: 0.9906
Epoch 17/100
 - 0s - loss: 0.0302 - accuracy: 0.9906
Epoch 18/100
 - 0s - loss: 0.0300 - accuracy: 0.9906
Epoch 19/100
 - 0s - loss: 0.0299 - accuracy: 0.9906
Epoch 20/100
 - 0s - loss: 0.0296 - accuracy: 0.9906
Epoch 21/100
 - 0s - loss: 0.0294 - accuracy: 0.9906
Epoch 22/100
 - 0s - loss: 0.0292 - accuracy: 0.9906
Epoch 23/100
 - 0s - loss: 0.0290 - accuracy: 0.9906
Epoch 24/100
 - 0s - loss: 0.0289 - accuracy: 0.9906
Epoch 25/100
 - 0s - loss: 0.0288 - accuracy: 0.9911
Epoch 26/100
 - 0s - loss: 0.0287 - accuracy: 0.9919
Epoch 27/100
 - 0s - loss: 0.0286 - accuracy: 0.9920
Epoch 28/100
 - 0s - loss: 0.0285 - accuracy: 0.9920
Epoch 29/100
 - 0s - loss: 0.0284 - accuracy: 

Epoch 66/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 67/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 68/100
 - 0s - loss: 0.0274 - accuracy: 0.9921
Epoch 69/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 70/100
 - 0s - loss: 0.0274 - accuracy: 0.9921
Epoch 71/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 72/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 73/100
 - 0s - loss: 0.0274 - accuracy: 0.9921
Epoch 74/100
 - 0s - loss: 0.0274 - accuracy: 0.9921
Epoch 75/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 76/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 77/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 78/100
 - 0s - loss: 0.0273 - accuracy: 0.9920
Epoch 79/100
 - 0s - loss: 0.0274 - accuracy: 0.9921
Epoch 80/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 81/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 82/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 83/100
 - 0s - loss: 0.0274 - accuracy: 0.9920
Epoch 84/100
 - 0s - loss: 0.0274 - accuracy: 

In [19]:
results

[(0.9618543196727065, 0.9697026811905274),
 (0.9671741710691755, 0.9456944310433553),
 (0.9625370360120782, 0.964848470183836),
 (0.9615949531886478, 0.9681117220966801),
 (0.9619808151945799, 0.9671611494118726)]

In [20]:
getResultSummary(results)

{'Train Mean': 0.9630282590274376,
 'Valid Mean': 0.9631036907852544,
 'Train Std': 0.002095687215580261,
 'Valid Std': 0.008845525449654734}

In [21]:
def saveStackingFile(prefix):
    oof_ = pd.DataFrame(oof,columns = [prefix])
    oot_ = pd.DataFrame(oot,columns = [prefix])
    print(oof_.shape, oot_.shape)
    oof_.to_csv(prefix + '_train.csv', index = False)
    oot_.to_csv(prefix + '_test.csv', index = False)

saveStackingFile('NN_level1')

(78369, 1) (50226, 1)
